In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity
import os

In [2]:
def build_feature_extractor(model_name):
    if model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False)
    elif model_name == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False)
    else:
        raise ValueError("Model not supported.")
    
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [3]:
feature_extractor_resnet = build_feature_extractor('resnet50')
feature_extractor_vgg16 = build_feature_extractor('vgg16')

In [4]:
def preprocess_resnet(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

def preprocess_vgg(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

In [5]:
# Database to store genuine signatures
signature_database_resnet = {}
signature_database_vgg16 = {}

# Store genuine signatures
def store_genuine_signature(user_id, img_path):
    # Preprocess for ResNet-50
    img_resnet = preprocess_resnet(img_path)
    features_resnet = feature_extractor_resnet.predict(img_resnet)
    signature_database_resnet[user_id] = features_resnet.reshape(1, -1)  # (1, 2048)

    # Preprocess for VGG-16
    img_vgg = preprocess_vgg(img_path)
    features_vgg = feature_extractor_vgg16.predict(img_vgg)
    signature_database_vgg16[user_id] = features_vgg.reshape(1, -1)  # (1, 512)

    print(f"Stored genuine signature for user {user_id}.")

In [6]:
def verify_signature(user_id, input_img_path, threshold=0.85):
    # Check if user exists
    if user_id not in signature_database_resnet or user_id not in signature_database_vgg16:
        raise ValueError("User not found.")

    # Load stored features
    stored_resnet = signature_database_resnet[user_id]  # (1, 2048)
    stored_vgg = signature_database_vgg16[user_id]      # (1, 512)

    # Preprocess input for both models
    input_resnet = preprocess_resnet(input_img_path)
    input_vgg = preprocess_vgg(input_img_path)

    # Extract features
    input_features_resnet = feature_extractor_resnet.predict(input_resnet).reshape(1, -1)
    input_features_vgg = feature_extractor_vgg16.predict(input_vgg).reshape(1, -1)

    # Compute similarities
    similarity_resnet = cosine_similarity(stored_resnet, input_features_resnet)[0][0]
    similarity_vgg = cosine_similarity(stored_vgg, input_features_vgg)[0][0]

    # Results
    resnet_result = "Genuine" if similarity_resnet >= threshold else "Forged"
    vgg_result = "Genuine" if similarity_vgg >= threshold else "Forged"

    print(f"ResNet-50: {resnet_result} (Score: {similarity_resnet:.2f})")
    print(f"VGG-16: {vgg_result} (Score: {similarity_vgg:.2f})")

In [7]:
store_genuine_signature('user_1', 'real_signatures/001001_001.png')
store_genuine_signature('user_2', 'real_signatures/001001_001.png')
verify_signature('user_1', 'forged_signatures/021001_000.png')
verify_signature('user_2', 'forged_signatures/021012_000.png')

1/1 [==============================] - 0s 299ms/step
Stored genuine signature for user user_1.
1/1 [==============================] - 0s 231ms/step
Stored genuine signature for user user_2.
1/1 [==============================] - 0s 127ms/step
ResNet-50: Genuine (Score: 0.92)
VGG-16: Genuine (Score: 0.89)
1/1 [==============================] - 0s 153ms/step
ResNet-50: Forged (Score: 0.72)
VGG-16: Forged (Score: 0.68)
